In [1]:
import pandas as pd
import numpy as np
import os

from datetime import timedelta
from gspread_dataframe import set_with_dataframe

import time
import warnings
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd

###### Import csv's

In [2]:
%%time

df_atributes0 = pd.read_csv('attributes.csv', delimiter=';')

df_city0 = pd.read_csv('city.csv', delimiter=';')

df_country0 = pd.read_csv('country.csv', delimiter=';')

df_location0 = pd.read_csv('location.csv', delimiter=';')

df_neighborhood0 = pd.read_csv('neighborhood.csv', delimiter=';')

df_results0 = pd.read_csv('results.csv', delimiter=';')

df_seller_contact0 = pd.read_csv('seller_contact.csv', delimiter=';')

df_state_u0 = pd.read_csv('state_u0.csv', delimiter=';')

Wall time: 1.88 s


In [ ]:
## Limpieza details products
df_atributes = df_atributes0[['__KEY_results','name_u6','value_name']]

df_km = df_atributes[df_atributes.name_u6=='Kilómetros'].rename(columns={'value_name':'Kilómetros'}).drop(['name_u6'],axis=1).reset_index(drop=True)
df_Marca = df_atributes[df_atributes.name_u6=='Marca'].rename(columns={'value_name':'Marca'}).drop(['name_u6'],axis=1).reset_index(drop=True)
df_Modelo = df_atributes[df_atributes.name_u6=='Modelo'].rename(columns={'value_name':'Modelo'}).drop(['name_u6'],axis=1).reset_index(drop=True)
df_Transmision = df_atributes[df_atributes.name_u6=='Transmisión'].rename(columns={'value_name':'Transmisión'}).drop(['name_u6'],axis=1).reset_index(drop=True)
df_year = df_atributes[df_atributes.name_u6=='Año'].rename(columns={'value_name':'Año'}).drop(['name_u6'],axis=1).reset_index(drop=True)
df_Version = df_atributes[df_atributes.name_u6=='Versión'].rename(columns={'value_name':'Versión'}).drop(['name_u6'],axis=1).reset_index(drop=True)

## Final DF
df_atributes1 = df_km.merge(df_Marca,on=['__KEY_results'],how='outer')
df_atributes1 = df_atributes1.merge(df_Modelo,on=['__KEY_results'],how='outer')
df_atributes1 = df_atributes1.merge(df_Transmision,on=['__KEY_results'],how='outer')
df_atributes1 = df_atributes1.merge(df_year,on=['__KEY_results'],how='outer')
df_atributes1 = df_atributes1.merge(df_Version,on=['__KEY_results'],how='outer')

In [13]:
df_atributes0.name_u6.drop_duplicates().to_list()

['Kilómetros', 'Marca', 'Modelo', 'Transmisión', 'Año', 'Versión']

In [ ]:
for i in df_atributes0.name_u6.drop_duplicates().to_list():
    

##### Armado de DF General

In [ ]:
df_product0 = df_results0.merge(df_atributes1,on=['__KEY_results'],how='left')

df_product0 = df_product0.merge(df_seller_contact0,on=['__KEY_results'],how='left')

df_product0 = df_product0.merge(df_location0,on=['__KEY_results'],how='left')

In [ ]:
%%time
clear1 = pd.DataFrame()

for i in df_product0.id_u9.unique():
    
    clear0 = df_product0[df_product0.id_u9==i].reset_index(drop=True)
    
    filter_clear0 = [clear0.__KEY_results.min(),clear0.__KEY_results.max()]
    
    clear = clear0[clear0.__KEY_results.isin(filter_clear0)].reset_index(drop=True)
    
    for n in range(0,len(clear)):
        try:
            clear['prev_price'] = clear['price'][n-1]
        except:
            pass

       
    clear1 = clear1.append(clear).drop_duplicates().reset_index(drop=True)
        

In [ ]:
### Limpieza
df_product = clear1[['__KEY_results', 'id_u9','accepts_mercadopago','available_quantity','condition',
       'currency_id', 'listing_type_id','official_store_id', 'price','prev_price','permalink',
      'site_id','sold_quantity', 'thumbnail', 'title','Kilómetros', 'Marca', 'Modelo', 'Transmisión', 
       'Año', 'Versión','area_code_u0','contact', 'email', 'phone','webpage', '__KEY_location', 'address_line_u0', 
       'subneighborhood', 'zip_code_u0']]

In [ ]:
%%time

df_product1 = pd.DataFrame()

for i in df_product.id_u9.unique():
    
    clear_last0 = df_product[df_product.id_u9==i].reset_index(drop=True)
    
    filter_last = [clear_last0.__KEY_results.max()]
    
    clear_last0 = clear_last0[clear_last0.__KEY_results.isin(filter_last)].reset_index(drop=True)
    
    
    df_product1 = df_product1.append(clear_last0).drop_duplicates().reset_index(drop=True)

In [ ]:
df_product1.loc[df_product1['Kilómetros']=='0','kilometraje'] = '0 KM'

df_product1.loc[(df_product1['Kilómetros']<='25000')&(df_product1['Kilómetros']>'0'),'kilometraje'] = '0 a 25.000 KM'

df_product1.loc[(df_product1['Kilómetros']>'25000')&(df_product1['Kilómetros']<='50000'),'kilometraje'] = '25.000 a 50.000 KM'

df_product1.loc[(df_product1['Kilómetros']>'50000')&(df_product1['Kilómetros']<='100000'),'kilometraje'] = '50.000 a 100.000 KM'

df_product1.loc[(df_product1['Kilómetros']>'100000')&(df_product1['Kilómetros']<'500000'),'kilometraje'] = '100.000 a 500.000 KM'

df_product1.loc[df_product1['Kilómetros']>'50000','kilometraje'] = 'Mas de 500.000 KM'

df_product1.loc[df_product1['Kilómetros']!=df_product1['Kilómetros'],'kilometraje'] = 'Sin Datos'

In [ ]:
len(df_product1)

In [ ]:
df_product2 =df_product1.merge(df_country0,on='__KEY_location',how='left')

len(df_product2)

In [ ]:
# Completo las marcas sin datos pero que aparecen en los titulos

df_product2.loc[(df_product2.Marca==df_product2.Marca),'Marca'] = df_product2.Marca


df_product2.loc[(df_product2.Marca!=df_product2.Marca)&(df_product2.title.apply(lambda x: x.lower()).str.contains('vols'))
                |(df_product2.title.apply(lambda x: x.lower()).str.contains('vw'))
                |(df_product2.title.apply(lambda x: x.lower()).str.contains('volk'))
                |(df_product2.title.apply(lambda x: x.lower()).str.contains('vento')),'Marca'] = 'Volkswagen'

df_product2.loc[(df_product2.Marca!=df_product2.Marca)&(df_product2.title.apply(lambda x: x.lower()).str.contains('suzu'))
                |(df_product2.title.apply(lambda x: x.lower()).str.contains('susu')),'Marca'] = 'Suzuki'

df_product2.loc[(df_product2.Marca!=df_product2.Marca)&
                (df_product2.title.apply(lambda x: x.lower()).str.contains('citro')),'Marca'] = 'Citroën'

df_product2.loc[(df_product2.Marca!=df_product2.Marca)
                &(df_product2.title.apply(lambda x: x.lower()).str.contains('peug')),'Marca'] = 'Peugeot'
 

In [ ]:
# Completo los modelos sin datos pero que aparecen en los titulos

df_product2.loc[(df_product2.Modelo==df_product2.Modelo),'Modelo'] = df_product2.Modelo


df_product2.loc[(df_product2.Modelo!=df_product2.Modelo)
                &(df_product2.title.apply(lambda x: x.lower()).str.contains('vento')),'Modelo'] = 'Vento'

df_product2.loc[(df_product2.Modelo!=df_product2.Modelo)
&(df_product2.title.apply(lambda x: x.lower()).str.contains('pica')),'Modelo'] = 'Picasso'

df_product2.loc[(df_product2.Modelo!=df_product2.Modelo)&(df_product2.title.apply(lambda x: x.lower()).str.contains('fun'))
                |(df_product2.title.apply(lambda x: x.lower()).str.contains('fum')),'Modelo'] = 'Fun'

In [ ]:
## Solo primer letra en cap resto min
df_product2.Modelo = df_product2.Modelo.str.capitalize()
df_product2.Marca = df_product2.Marca.str.capitalize()

# Completar Campos vacios
df_product2.Año = df_product2.Año.fillna('Sin Datos')

df_product2['Transmisión'] = df_product2['Transmisión'].fillna('Sin Datos')

df_product2.Versión = df_product2.Versión.fillna('Sin Datos')

In [ ]:
## cambio en el precio de la primer a ultima modificacion de la publicacion
df_product2['cambio_precio'] = df_product2.price - df_product2.prev_price

In [ ]:
df_product2.loc[(df_product2.condition=='used'),'Condición'] = 'Usado'

df_product2.loc[(df_product2.condition=='not_specified'),'Condición'] = 'Sin Especificar'

df_product2.loc[(df_product2.condition=='new'),'Condición'] = 'Nuevo'

df_product2.loc[(df_product2.condition!=df_product2.condition),'Condición'] = 'Sin Datos'

In [ ]:
df_product2.to_csv('raw_data.csv',index=False)